In [3]:
from lxml import etree, html
import pandas as pd
from datetime import date
import requests

In [4]:
doc = etree.parse('BOE-A-2020-14372.xml')

In [5]:
#etree.tostring(doc,pretty_print=True ,xml_declaration=True, encoding="utf-8")

In [6]:
raiz=doc.getroot()
raiz.tag

'documento'

In [7]:
len(raiz)

4

In [8]:
libro=raiz[0]
libro.tag

'metadatos'

In [9]:
libro=raiz[0]
libro[0].text

'BOE-A-2020-14372'

In [10]:
libro=raiz[1]
libro[0].tag

'{http://www.w3.org/1999/02/22-rdf-syntax-ns#}RDF'

In [11]:
libro=raiz[1]
libro[0].text

'\n            '

In [12]:
Identificador=raiz.find("metadatos/identificador")
Titulo=raiz.find("metadatos/titulo")
URL_pdf=raiz.find("metadatos/url_pdf")

In [13]:
print(Identificador.text)
print(Titulo.text)
print("https://www.boe.es" + URL_pdf.text)


BOE-A-2020-14372
Decreto-ley 35/2020, de 20 de octubre, de modificación del Decreto-ley 19/2020, de 19 de mayo, y del Decreto-ley 29/2020, de 28 de julio, en materia de adopción de medidas sociales y sanitarias para paliar los efectos de la pandemia generada por la COVID-19.
https://www.boe.es/boe/dias/2020/11/18/pdfs/BOE-A-2020-14372.pdf


In [14]:
materia=raiz.find("analisis/materias/materia")
materia.text

'Asistencia sanitaria'

In [15]:
materias=raiz.findall("analisis/materias/materia")
for materia in materias:
    print(materia.text)

Asistencia sanitaria
Asistencia social
Ayudas
Cataluña
Centros residenciales para personas con discapacidad
Epidemias
Establecimientos residenciales para la Tercera Edad
Pagos
Servicios Sociales de las Comunidades Autónomas


In [16]:
type(materias)

list

In [17]:
alertas=raiz.findall("analisis/alertas/alerta")
for alerta in alertas:
    print(alerta.text)

Asuntos sociales
Sanidad


# Recoge resumen diario del BOE de hoy

In [18]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)



Fecha de Hoy = 20201223
dia = 23
mes = 12
año = 2020


In [19]:
URL_XML_resumen =  "https://www.boe.es/diario_boe/xml.php?id=BOE-S-" + str(hoy)

In [20]:
URL_XML_resumen

'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20201223'

In [21]:
#pip install requests for installation



url = URL_XML_resumen
r = requests.get(url)
#r.content
#r.text

In [22]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)
        
#save_html(r.content, './BOEs/resumen.xml')
save_html(r.content, './BOEs/Resumen-BOE-' + hoy + '.xml')

In [23]:
#r.encoding

In [24]:
#resumen = etree.parse('./BOEs/resumen.xml')
resumen = etree.parse('./BOEs/Resumen-BOE-' + hoy + '.xml')

In [25]:
raiz=resumen.getroot()
raiz.tag
raiz_sumario = raiz

In [26]:
seccion = raiz.findall("sumario/diario/seccion")
seccion

[]

In [27]:
for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')
    print(nombre_seccion)

['I. Disposiciones generales']
['II. Autoridades y personal. - A. Nombramientos, situaciones e incidencias']
['II. Autoridades y personal. - B. Oposiciones y concursos']
['III. Otras disposiciones']
['IV. Administración de Justicia']
['V. Anuncios. - A. Contratación del Sector Público']
['V. Anuncios. - B. Otros anuncios oficiales']


In [28]:
secciones = raiz.findall("sumario/diario/seccion")
for seccion in secciones:
    print(seccion.text)

In [29]:
tabla_resumen = pd.DataFrame()

In [30]:
#for seccion in raiz.xpath('//seccion[contains(@nombre, "III. Otras disposiciones") or contains(@nombre, "I. Disposiciones generales")]'):
for seccion in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales")]'):
#for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')

    for departamento in seccion:
        nombre_departamento = departamento.xpath('@nombre') 

        for epigrafe in departamento:
            nombre_epigrafe = epigrafe.xpath('@nombre')

            for item in epigrafe:
                item_id = item.xpath('@id')
                item_name = item.xpath('.//titulo/text()') 

                item_urlXml = "https://www.boe.es" + str(item.xpath('.//urlXml/text()'))[2:-2]
                #print(item_urlXml)

                tabla_resumen = tabla_resumen.append({'Seccion': nombre_seccion, 
                                                      'Departamento': nombre_departamento, 
                                                      'Epigrafe' : nombre_epigrafe,
                                                      'Item_id': item_id, 
                                                      'Item_Nombre' : item_name, 
                                                      'Item_URL_XML' : item_urlXml},
                                                      ignore_index=True)


In [31]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[JEFATURA DEL ESTADO],[Medidas urgentes],"[Real Decreto-ley 35/2020, de 22 de diciembre,...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16823],[I. Disposiciones generales]
1,[JEFATURA DEL ESTADO],[Medidas urgentes],"[Real Decreto-ley 37/2020, de 22 de diciembre,...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16824],[I. Disposiciones generales]
2,[CORTES GENERALES],[Medidas urgentes],"[Resolución de 17 de diciembre de 2020, del Co...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16825],[I. Disposiciones generales]
3,"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Tratados internacionales],[Textos enmendados aprobados en París el 15 de...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16826],[I. Disposiciones generales]
4,[MINISTERIO DE HACIENDA],[Organización],"[Orden HAC/1238/2020, de 17 de noviembre de 20...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16827],[I. Disposiciones generales]
5,[MINISTERIO DE HACIENDA],[Incentivos regionales],"[Orden HAC/1239/2020, de 21 de diciembre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16828],[I. Disposiciones generales]
6,[MINISTERIO DE HACIENDA],[Comercio intracomunitario. Estadísticas],"[Resolución de 15 de diciembre de 2020, del De...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16829],[I. Disposiciones generales]
7,"[MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA...",[Red ferroviaria],"[Orden TMA/1240/2020, de 8 de diciembre, por l...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16830],[I. Disposiciones generales]
8,"[MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA...",[Seguridad aérea],"[Resolución de 17 de diciembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16831],[I. Disposiciones generales]
9,"[MINISTERIO DE INDUSTRIA, COMERCIO Y TURISMO]",[Subvenciones],"[Real Decreto 1153/2020, de 22 de diciembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16832],[I. Disposiciones generales]


# Descarga ficheros XML asociados

In [34]:
for item_URL in tabla_resumen['Item_URL_XML']:
    #print(URL)
    r = requests.get(item_URL)
    f = './BOEs/' + item_URL[-16:] + '.xml'
    save_html(r.content, f)

In [35]:

import glob
BOEs = glob.glob('./BOEs/BOE*.xml')
print (BOEs)

['./BOEs/BOE-A-2020-15394.xml', './BOEs/BOE-A-2020-16676.xml', './BOEs/BOE-A-2020-14465.xml', './BOEs/BOE-A-2020-14459.xml', './BOEs/BOE-A-2020-15237.xml', './BOEs/BOE-A-2020-15223.xml', './BOEs/BOE-A-2020-15222.xml', './BOEs/BOE-A-2020-15236.xml', './BOEs/BOE-A-2020-14458.xml', './BOEs/BOE-A-2020-14464.xml', './BOEs/BOE-A-2020-15395.xml', './BOEs/BOE-A-2020-15397.xml', './BOEs/BOE-A-2020-16675.xml', './BOEs/BOE-A-2020-14466.xml', './BOEs/BOE-A-2020-15208.xml', './BOEs/BOE-A-2020-14880.xml', './BOEs/BOE-A-2020-15220.xml', './BOEs/BOE-A-2020-15234.xml', './BOEs/BOE-A-2020-15235.xml', './BOEs/BOE-A-2020-15221.xml', './BOEs/BOE-A-2020-15209.xml', './BOEs/BOE-A-2020-14467.xml', './BOEs/BOE-A-2020-16674.xml', './BOEs/BOE-A-2020-15396.xml', './BOEs/BOE-A-2020-15392.xml', './BOEs/BOE-A-2020-16843.xml', './BOEs/BOE-A-2020-16670.xml', './BOEs/BOE-A-2020-14463.xml', './BOEs/BOE-A-2020-15225.xml', './BOEs/BOE-A-2020-15231.xml', './BOEs/BOE-A-2020-15219.xml', './BOEs/BOE-A-2020-15218.xml', './BOEs

In [36]:
#import os
#path = './BOEs'
#BOEs = [f for f in os.listdir(path) if f.startswith('BOE')]
#BOEs

In [37]:
tabla_materias = pd.DataFrame()

for BOE in BOEs:
    #print (BOE)
    BOE_XML = etree.parse(BOE)
    raiz=BOE_XML.getroot()

    materias = [materia.text for materia in raiz.findall('analisis/materias/materia')]
    alertas = [alerta.text for alerta in raiz.findall('analisis/alertas/alerta')]
    #Item_Id = '[' + BOE[7:23] + ']'
    #Item_Id = raiz.find('metadatos/identificador')
    Item_Id = raiz.xpath(".//identificador/text()")
 
    #print(Item_Id, materias, alertas)
    #tabla_materias.loc[['Item_id'], ['Materias']] = materias
    tabla_materias = tabla_materias.append({'Item_id': Item_Id,
                                            'Materias' : materias,
                                            'Alertas' : alertas},
                                            ignore_index=True)

    #tabla_materias.Item_id.apply(list)
    #fila_tabla = tabla_resumen.set_index('Item_id').filter(like=BOE[18:23], axis=0).reset_index()
    #print(fila_tabla['Item_id'])

    #tabla_resumen.loc[fila_tabla['Item_id']]['Materias'] = materias


In [38]:
tabla_materias.sort_values('Item_id')

,Alertas,Item_id,Materias
175,"[Relaciones internacionales, Sanidad, Sistema ...",[BOE-A-2020-14457],"[Acuerdos internacionales, Créditos Presupuest..."
8,"[Educación y enseñanza, Relaciones internacion...",[BOE-A-2020-14458],"[Acuerdos internacionales, Barcelona, Centros ..."
3,"[Educación y enseñanza, Relaciones internacion...",[BOE-A-2020-14459],"[Acuerdos internacionales, Barcelona, Centros ..."
41,"[Administración de Justicia, Sistema financiero]",[BOE-A-2020-14460],"[Ayuntamientos, Créditos Presupuestarios, Juzg..."
48,[Sistema financiero],[BOE-A-2020-14461],[Administración y Contabilidad de la Hacienda ...
...,...,...,...
136,"[Medio ambiente, Obras y construcciones]",[BOE-A-2020-16839],"[Abastecimiento de aguas, Ayudas, Canarias, Ob..."
39,"[Agricultura, Medio ambiente]",[BOE-A-2020-16840],"[Abastecimiento de aguas, Agricultura, Aguas, ..."
50,[Energía],[BOE-A-2020-16841],"[Almacenes, Carburantes y combustibles, Corpor..."
36,"[Deporte, Sanidad, Sistema financiero]",[BOE-A-2020-16842],"[Asociaciones deportivas, Aval, Ayudas, Contab..."


In [39]:
print(raiz.tag)

documento


In [40]:
for materias in raiz.xpath('.//materias'):
    materia = seccion.xpath('.//materia/text()')
    print(materia)

[]


In [41]:
materias=raiz.findall("analisis/materias/materia")
for materia in materias:
    print(materia.text)

Asociaciones
Fundaciones
Medio ambiente
Ministerio para la Transición Ecológica y el Reto Demográfico
Subvenciones


In [42]:
alertas=raiz.findall("analisis/alertas/alerta")
for alerta in alertas:
    print(alerta.text)

Medio ambiente


In [43]:
#tabla_resumen.loc[tabla_resumen['Item_id'] == '[' + BOE[7:23] + ']']
print(tabla_resumen.loc[tabla_resumen['Item_id'] == '[BOE-A-2020-14457]'])

Empty DataFrame
Columns: [Departamento, Epigrafe, Item_Nombre, Item_URL_XML, Item_id, Seccion]
Index: []


In [44]:
BOE[7:23]

'BOE-A-2020-15399'

In [45]:
tabla_resumen['Item_id']

0     [BOE-A-2020-16823]
1     [BOE-A-2020-16824]
2     [BOE-A-2020-16825]
3     [BOE-A-2020-16826]
4     [BOE-A-2020-16827]
5     [BOE-A-2020-16828]
6     [BOE-A-2020-16829]
7     [BOE-A-2020-16830]
8     [BOE-A-2020-16831]
9     [BOE-A-2020-16832]
10    [BOE-A-2020-16833]
11    [BOE-A-2020-16834]
12    [BOE-A-2020-16835]
13    [BOE-A-2020-16836]
14    [BOE-A-2020-16837]
15    [BOE-A-2020-16838]
16    [BOE-A-2020-16839]
17    [BOE-A-2020-16840]
18    [BOE-A-2020-16841]
19    [BOE-A-2020-16842]
20    [BOE-A-2020-16843]
Name: Item_id, dtype: object

In [46]:
tabla_materias['Item_id']

0      [BOE-A-2020-15394]
1      [BOE-A-2020-16676]
2      [BOE-A-2020-14465]
3      [BOE-A-2020-14459]
4      [BOE-A-2020-15237]
              ...        
180    [BOE-A-2020-15207]
181    [BOE-A-2020-15206]
182    [BOE-A-2020-15212]
183    [BOE-A-2020-15400]
184    [BOE-A-2020-15399]
Name: Item_id, Length: 185, dtype: object

In [47]:
tabla_resumen.dtypes

Departamento    object
Epigrafe        object
Item_Nombre     object
Item_URL_XML    object
Item_id         object
Seccion         object
dtype: object

In [48]:
tabla_resumen.set_index('Item_id').filter(like='14875', axis=0)

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Seccion
Item_id,,,,,


In [49]:
#tabla_resumen = tabla_resumen.astype(str)
#tabla_resumen['Joined'] = tabla_resumen.Item_id.apply(', '.join)
#tabla_resumen['Item_id']=tabla_resumen['Item_id']

In [50]:
tabla_resumen.dtypes

Departamento    object
Epigrafe        object
Item_Nombre     object
Item_URL_XML    object
Item_id         object
Seccion         object
dtype: object

In [51]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[JEFATURA DEL ESTADO],[Medidas urgentes],"[Real Decreto-ley 35/2020, de 22 de diciembre,...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16823],[I. Disposiciones generales]
1,[JEFATURA DEL ESTADO],[Medidas urgentes],"[Real Decreto-ley 37/2020, de 22 de diciembre,...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16824],[I. Disposiciones generales]
2,[CORTES GENERALES],[Medidas urgentes],"[Resolución de 17 de diciembre de 2020, del Co...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16825],[I. Disposiciones generales]
3,"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Tratados internacionales],[Textos enmendados aprobados en París el 15 de...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16826],[I. Disposiciones generales]
4,[MINISTERIO DE HACIENDA],[Organización],"[Orden HAC/1238/2020, de 17 de noviembre de 20...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16827],[I. Disposiciones generales]
5,[MINISTERIO DE HACIENDA],[Incentivos regionales],"[Orden HAC/1239/2020, de 21 de diciembre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16828],[I. Disposiciones generales]
6,[MINISTERIO DE HACIENDA],[Comercio intracomunitario. Estadísticas],"[Resolución de 15 de diciembre de 2020, del De...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16829],[I. Disposiciones generales]
7,"[MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA...",[Red ferroviaria],"[Orden TMA/1240/2020, de 8 de diciembre, por l...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16830],[I. Disposiciones generales]
8,"[MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA...",[Seguridad aérea],"[Resolución de 17 de diciembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16831],[I. Disposiciones generales]
9,"[MINISTERIO DE INDUSTRIA, COMERCIO Y TURISMO]",[Subvenciones],"[Real Decreto 1153/2020, de 22 de diciembre, p...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2020-16832],[I. Disposiciones generales]


In [52]:
#for row in tabla_resumen:
#    tabla_resumen.loc[row['Item_id']]['Materias'] = tabla_materias.loc[row['Item_id']]['Materias']

In [53]:
tabla_resumen['Item_id'] = tabla_resumen['Item_id'].astype(str)
tabla_materias['Item_id'] = tabla_materias['Item_id'].astype(str)
tabla_resultados = tabla_resumen.set_index('Item_id').join(tabla_materias.set_index('Item_id'))


In [54]:
tabla_resultados

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Seccion,Alertas,Materias
Item_id,,,,,,,
['BOE-A-2020-16823'],[JEFATURA DEL ESTADO],[Medidas urgentes],"[Real Decreto-ley 35/2020, de 22 de diciembre,...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Asuntos sociales, Comercio, Cultura y ocio, D...","[Actividades económicas, Arrendamientos urbano..."
['BOE-A-2020-16824'],[JEFATURA DEL ESTADO],[Medidas urgentes],"[Real Decreto-ley 37/2020, de 22 de diciembre,...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Asuntos sociales, Consumidores y usuarios, De...","[Aguas, Asistencia social, Ayudas, Consumidore..."
['BOE-A-2020-16825'],[CORTES GENERALES],[Medidas urgentes],"[Resolución de 17 de diciembre de 2020, del Co...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Comercio, Derecho Mercantil, Energía, Industr...","[Aranceles Profesionales, Asistencia sanitaria..."
['BOE-A-2020-16826'],"[MINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EUROP...",[Tratados internacionales],[Textos enmendados aprobados en París el 15 de...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Deporte, Relaciones internacionales]","[Acuerdos internacionales, Autorizaciones, Dep..."
['BOE-A-2020-16827'],[MINISTERIO DE HACIENDA],[Organización],"[Orden HAC/1238/2020, de 17 de noviembre de 20...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Derecho Administrativo, Organización de la Ad...","[Juntas de Contratación, Mesas de Contratación..."
['BOE-A-2020-16828'],[MINISTERIO DE HACIENDA],[Incentivos regionales],"[Orden HAC/1239/2020, de 21 de diciembre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Organización de la Administración, Sistema tr...","[Comunidades Autónomas, Financiación de las Co..."
['BOE-A-2020-16829'],[MINISTERIO DE HACIENDA],[Comercio intracomunitario. Estadísticas],"[Resolución de 15 de diciembre de 2020, del De...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Comercio, Unión Europea]",[Agencia Estatal de la Administración Tributar...
['BOE-A-2020-16830'],"[MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA...",[Red ferroviaria],"[Orden TMA/1240/2020, de 8 de diciembre, por l...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],[Transportes y tráfico],"[Ferrocarriles, Madrid, Red Ferroviaria de Int..."
['BOE-A-2020-16831'],"[MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA...",[Seguridad aérea],"[Resolución de 17 de diciembre de 2020, de la ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[I. Disposiciones generales],"[Consumidores y usuarios, Derecho Administrati...","[Distintivos, Enfermedades, Epidemias, Formula..."


In [55]:
#tabla_resultados.to_csv('./BOEs/Resultados.csv', index=False)
tabla_resultados.to_csv('./BOEs/Resultados-BOE-' + hoy + '.csv', index=False)

## Genera Split de ALertas y crea una fila para cada una

### Los items sin Alerta se desechan

In [56]:
tabla_resultados.Alertas.apply(pd.Series) \
    .merge(tabla_resultados, left_index = True, right_index = True) \
    .drop(["Alertas"], axis = 1) \
    .reset_index() \
    .melt(id_vars = ['Item_id','Item_Nombre','Seccion','Departamento','Epigrafe','Item_URL_XML','Materias',], value_name = "Alerta") \
    .drop("variable", axis = 1) \
    .dropna() \
    .sort_values('Alerta')

,Item_id,Item_Nombre,Seccion,Departamento,Epigrafe,Item_URL_XML,Materias,Alerta
12,['BOE-A-2020-16835'],"[Real Decreto 1156/2020, de 22 de diciembre, d...",[I. Disposiciones generales],[MINISTERIO DE POLÍTICA TERRITORIAL Y FUNCIÓN ...,[Comunidad Autónoma de Cataluña. Traspaso de f...,https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Administración de Justicia, Cataluña, Comunid...",Administración de Justicia
17,['BOE-A-2020-16840'],"[Real Decreto 1161/2020, de 22 de diciembre, p...",[I. Disposiciones generales],[MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL ...,[Subvenciones],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Abastecimiento de aguas, Agricultura, Aguas, ...",Agricultura
20,['BOE-A-2020-16843'],"[Real Decreto 1163/2020, de 22 de diciembre, p...",[I. Disposiciones generales],[MINISTERIO DE DERECHOS SOCIALES Y AGENDA 2030],[Subvenciones],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Administración Local, Asistencia social, Ayud...",Asuntos sociales
0,['BOE-A-2020-16823'],"[Real Decreto-ley 35/2020, de 22 de diciembre,...",[I. Disposiciones generales],[JEFATURA DEL ESTADO],[Medidas urgentes],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Actividades económicas, Arrendamientos urbano...",Asuntos sociales
1,['BOE-A-2020-16824'],"[Real Decreto-ley 37/2020, de 22 de diciembre,...",[I. Disposiciones generales],[JEFATURA DEL ESTADO],[Medidas urgentes],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Aguas, Asistencia social, Ayudas, Consumidore...",Asuntos sociales
...,...,...,...,...,...,...,...,...
71,['BOE-A-2020-16831'],"[Resolución de 17 de diciembre de 2020, de la ...",[I. Disposiciones generales],"[MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA...",[Seguridad aérea],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Distintivos, Enfermedades, Epidemias, Formula...",Transportes y tráfico
191,['BOE-A-2020-16825'],"[Resolución de 17 de diciembre de 2020, del Co...",[I. Disposiciones generales],[CORTES GENERALES],[Medidas urgentes],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Aranceles Profesionales, Asistencia sanitaria...",Transportes y tráfico
189,['BOE-A-2020-16823'],"[Real Decreto-ley 35/2020, de 22 de diciembre,...",[I. Disposiciones generales],[JEFATURA DEL ESTADO],[Medidas urgentes],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,"[Actividades económicas, Arrendamientos urbano...",Turismo
27,['BOE-A-2020-16829'],"[Resolución de 15 de diciembre de 2020, del De...",[I. Disposiciones generales],[MINISTERIO DE HACIENDA],[Comercio intracomunitario. Estadísticas],https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[Agencia Estatal de la Administración Tributar...,Unión Europea


In [57]:
tabla_resultados['Epigrafe'].str.join(',')

Item_id
['BOE-A-2020-16823']                                     Medidas urgentes
['BOE-A-2020-16824']                                     Medidas urgentes
['BOE-A-2020-16825']                                     Medidas urgentes
['BOE-A-2020-16826']                             Tratados internacionales
['BOE-A-2020-16827']                                         Organización
['BOE-A-2020-16828']                                Incentivos regionales
['BOE-A-2020-16829']              Comercio intracomunitario. Estadísticas
['BOE-A-2020-16830']                                      Red ferroviaria
['BOE-A-2020-16831']                                      Seguridad aérea
['BOE-A-2020-16832']                                         Subvenciones
['BOE-A-2020-16833']                        Riesgos laborales. Prevención
['BOE-A-2020-16834']    Medicamentos y productos sanitarios. Fuerzas A...
['BOE-A-2020-16835']    Comunidad Autónoma de Cataluña. Traspaso de fu...
['BOE-A-2020-16836']    Fondo 

In [58]:
from pprint import pprint

## Convierte estructura XML en diccionario

In [59]:
BOE_XML = etree.parse('./BOEs/BOE-A-2020-14457.xml')
raiz=BOE_XML.getroot()

In [60]:
d = [{x.tag: x.text for x in c} for c in raiz]
pprint(d)

[{'departamento': 'Ministerio de Asuntos Exteriores, Unión Europea y '
                  'Cooperación',
  'diario': 'Boletín Oficial del Estado',
  'diario_numero': '304',
  'estado_consolidacion': None,
  'estatus_derogacion': 'N',
  'estatus_legislativo': 'L',
  'fecha_derogacion': None,
  'fecha_disposicion': '20201030',
  'fecha_publicacion': '20201119',
  'fecha_vigencia': '20201101',
  'identificador': 'BOE-A-2020-14457',
  'judicialmente_anulada': 'N',
  'letra_imagen': 'A',
  'numero_oficial': None,
  'origen_legislativo': 'Estatal',
  'pagina_final': '101023',
  'pagina_inicial': '101017',
  'rango': 'Acuerdo Internacional',
  'seccion': '1',
  'subseccion': None,
  'suplemento_letra_imagen': None,
  'suplemento_pagina_final': None,
  'suplemento_pagina_inicial': None,
  'titulo': 'Acuerdo Internacional Administrativo, para el periodo 2020-2021, '
            'entre el Ministerio de Sanidad del Reino de España y la '
            'Organización Mundial de la Salud, para llevar a

In [61]:
pprint(raiz[2].tag)

'texto'


In [62]:
parrafos=raiz.findall("texto/p")
for parrafo in parrafos:
    print(parrafo.text)

ACUERDO INTERNACIONAL ADMINISTRATIVO, PARA EL PERÍODO 2020-2021, ENTRE EL MINISTERIO DE SANIDAD DEL REINO DE ESPAÑA Y LA ORGANIZACIÓN MUNDIAL DE LA SALUD, PARA LLEVAR A CABO ACTIVIDADES DE COOPERACIÓN EN MATERIA DE DONACIÓN Y TRASPLANTE DE ÓRGANOS, CÉLULAS Y TEJIDOS HUMANOS AL AMPARO DEL ACUERDO MARCO DE COOPERACIÓN ENTRE EL REINO DE ESPAÑA Y LA ORGANIZACIÓN MUNDIAL DE LA SALUD, DE 12 DE SEPTIEMBRE DE 2001
Signatarios:
Salvador Illa Roca, Ministro de Sanidad del Reino de España, en virtud del Real Decreto 8/2020, de 12 de enero, por el que se dispone su nombramiento, y en virtud de las facultades otorgadas en el artículo 48.2 de la Ley 40/2015, de 1 de octubre, de Régimen Jurídico del Sector Público.
Mariângela Simão, Subdirectora General de Acceso a Medicamentos, Vacunas y Productos Farmacéuticos de la Organización Mundial de la Salud (en adelante, la OMS), en representación de dicha Organización.
Los signatarios suscriben el presente documento en el marco jurídico del Acuerdo Marco d

# DOGC

In [63]:
URL_HTML_resumen =  "https://dogc.gencat.cat/es/index.html?newLang=es_ES&language=es_ES"

In [64]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get(URL_HTML_resumen)

In [65]:
r.html.links

{'/ca/index.html?newLang=es_ES&language=es_ES',
 '/ca/pdogc_serveis/cerca-republica/',
 '/ca/pdogc_serveis/pdogc_dogc_mobi/',
 '/es/ajuda/mapaweb/',
 '/es/index.html?newLang=es_ES&language=es_ES',
 '/es/pdogc_base_de_dades_del_dogc/',
 '/es/pdogc_canals_interns/pdogc_cercador_de_normativa/?destParam=cercaAdv',
 '/es/pdogc_canals_interns/pdogc_que_es_el_codi_de_verificacio/',
 '/es/pdogc_eadop/',
 '/es/pdogc_enllacos/',
 '/es/pdogc_preguntes_frequents/',
 '/es/pdogc_serveis/',
 '/es/pdogc_serveis/pdogc_normes_urbanistiques_1985-2002/',
 '/es/pdogc_serveis/pdogc_serveis_de_informacio/',
 '/es/pdogc_serveis/pdogc_serveis_de_informacio/pdogc_alertes_dogc/',
 '/es/pdogc_serveis/pdogc_serveis_de_publicacio_al_dogc/Publicacio_Anuncis_situacio_CoV-2/',
 '/es/pdogc_serveis/pdogc_serveis_de_publicacio_al_dogc/pdogc_dogc_eacat/index.html',
 '/es/pdogc_sobre_el_dogc/',
 'http://bop.diba.es/',
 'http://llibreria.gencat.cat/index.php?language=es',
 'http://web.gencat.cat/es/temes/',
 'http://www.boe

In [66]:
r.html.xpath('//*[@id="titol"]/span/text()')

[' DOGC núm.\r\n\t\t\t\t\t 8302 (23/12/2020)']

In [67]:
r.html.xpath('//*[@id="titol"]/h3/text()')


['\r\n\t\t\t\t\tÚltimo DOGC publicado\r\n\t\t\t\t']

In [68]:
r.html.lxml.tag

'html'

In [69]:
r.html.lxml.xpath('//*[@id="titol"]/span')

[<Element span at 0x7fbefc8dabd0>]

In [70]:
#import requests
#from lxml import html

response = requests.get(URL_HTML_resumen)

tree = html.fromstring(response.text)

# fetches all useful elements 
useful = tree.xpath('//*[@id="titol"]/span/text()')

print(useful)


[' DOGC núm.\r\n\t\t\t\t\t 8302 (23/12/2020)']


In [71]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

['/es/pdogc_canals_interns/pdogc_sumari_del_dogc/']

In [72]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@name')

['anexos']

In [73]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@value')

['1']

In [74]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form')

[<Element form at 0x7fbef9ae74f0>]

In [75]:
useful = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/input[3]/@value')

print(useful)

['8302']


In [76]:
tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

['/es/pdogc_canals_interns/pdogc_sumari_del_dogc/']

In [77]:
cadena = ''
for input in tree.xpath('//*[@id="sumari"]/ul/li[1]/form'):
    argumentos = input.xpath('./input/@name')
    valores = input.xpath('./input/@value')
    print(argumentos, valores)

for indice in range(len(argumentos)):
    cadena += argumentos[indice] + '=' + valores[indice] + '&'

argumentos[1]
cadena[:-1]

['anexos', 'language', 'numDOGC', 'seccion'] ['1', 'es_ES', '8302', '0']


'anexos=1&language=es_ES&numDOGC=8302&seccion=0'

## Recoge Nombre Secciones Sumario

In [78]:
import re
seccion = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/a/text()')
seccion = re.sub('(\\r|\\n|\\t|\ )+', '', seccion[0])
seccion

'DISPOSICIONES'

## Recoge Nombre Secciones Sumario

In [79]:
secciones = tree.xpath('//*[@id="sumari"]/ul/li')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion = seccion.xpath('./form/a/text()')
    seccion = re.sub('(\\r|\\n|\\t)+', '', seccion[0])
    print(seccion[1:-1])
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion[1:-1]}, ignore_index=True)


   DISPOSICIONES 
   CARGOS Y PERSONAL
   ANUNCIOS DE LA GENERALIDAD DE CATALUÑA
   ANUNCIOS DE LA ADMINISTRACIÓN LOCAL
   ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA 
   ANUNCIOS VARIOS


In [80]:
df_secciones_sumarios

,Seccion
0,DISPOSICIONES
1,CARGOS Y PERSONAL
2,ANUNCIOS DE LA GENERALIDAD DE CATALUÑA
3,ANUNCIOS DE LA ADMINISTRACIÓN LOCAL
4,ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA
5,ANUNCIOS VARIOS


## Recoge Valores para formar URLs Secciones Sumario

In [81]:
URL_base_sumario = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

df_URL_sumarios = pd.DataFrame()
for seccion in secciones:    
    cadena = ''
    for input in seccion.xpath('./form'):
        argumentos = input.xpath('./input/@name')
        valores = input.xpath('./input/@value')
        #print(argumentos, valores)

    for indice in range(len(argumentos)):
        cadena += argumentos[indice] + '=' + valores[indice] + '&'

    URL_sumario = 'https://dogc.gencat.cat' + str(URL_base_sumario[0]) + '?' + str(cadena[:-1])
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8302&seccion=0
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8302&seccion=1
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8302&seccion=2
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8302&seccion=3
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8302&seccion=4
https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8302&seccion=5


In [82]:
df_URL_sumarios

,URL_Seccion
0,https://dogc.gencat.cat/es/pdogc_canals_intern...
1,https://dogc.gencat.cat/es/pdogc_canals_intern...
2,https://dogc.gencat.cat/es/pdogc_canals_intern...
3,https://dogc.gencat.cat/es/pdogc_canals_intern...
4,https://dogc.gencat.cat/es/pdogc_canals_intern...
5,https://dogc.gencat.cat/es/pdogc_canals_intern...


In [83]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [84]:
df_sumarios

,Seccion,URL_Seccion
0,DISPOSICIONES,https://dogc.gencat.cat/es/pdogc_canals_intern...
1,CARGOS Y PERSONAL,https://dogc.gencat.cat/es/pdogc_canals_intern...
2,ANUNCIOS DE LA GENERALIDAD DE CATALUÑA,https://dogc.gencat.cat/es/pdogc_canals_intern...
3,ANUNCIOS DE LA ADMINISTRACIÓN LOCAL,https://dogc.gencat.cat/es/pdogc_canals_intern...
4,ANUNCIOS DE LA ADMINISTRACIÓN DE JUSTICIA,https://dogc.gencat.cat/es/pdogc_canals_intern...
5,ANUNCIOS VARIOS,https://dogc.gencat.cat/es/pdogc_canals_intern...


In [85]:
df_sumarios['URL_Seccion'][4]

'https://dogc.gencat.cat/es/pdogc_canals_interns/pdogc_sumari_del_dogc/?anexos=1&language=es_ES&numDOGC=8302&seccion=4'

## Recoge Items en Seccion Disposiciones

In [86]:
### Recoge Items en Seccion Disposiciones

response = requests.get(df_sumarios['URL_Seccion'][0])
sumario_HTML = html.fromstring(response.text)

#seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div[1]/text()')
seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div/text()')
bloques = sumario_HTML.xpath('//*[@id="disposicions"]')
df_disposiciones = pd.DataFrame(columns=['Seccion','Item_Name','PDF_link'])
for bloque in bloques: 
    item_name = bloque.xpath('./div/p/text()')
    #item_name = re.sub('(\\r|\\n|\\t)+', '', item_name[0])
    pdf_link = bloque.xpath('./div/div/a[4]/@href')

for row in range(len(pdf_link)):
    df_disposiciones = df_disposiciones.append({'Item_Name': re.sub('(\\r|\\n|\\t)+', '',                                                                               item_name[row]),
                                            'PDF_link' : pdf_link[row],
                                            'Seccion' : seccion[0][:-1]},
                                            ignore_index=True)

df_disposiciones

,Seccion,Item_Name,PDF_link
0,DISPOSICIONES,"RESOLUCIÓN de 14 de diciembre de 2020, de dele...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
1,DISPOSICIONES,"DICTAMEN 6/2019, de 30 de diciembre, sobre el ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
2,DISPOSICIONES,"RESOLUCIÓN de 16 de diciembre de 2020, por la ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
3,DISPOSICIONES,...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
4,DISPOSICIONES,"DECRETO LEY 52/2020, de 22 de diciembre, de re...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
5,DISPOSICIONES,...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
6,DISPOSICIONES,...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
7,DISPOSICIONES,"DECRETO 148/2020, de 22 de diciembre, de norma...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
8,DISPOSICIONES,"DECRETO 149/2020, de 22 de diciembre, por el q...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...
9,DISPOSICIONES,"RESOLUCIÓN EDU/3339/2020, de 17 de diciembre, ...",https://portaldogc.gencat.cat/utilsEADOP/AppJa...


In [87]:
pdf_link

['https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889570&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889581&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889670&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889595&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889636&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889658&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889664&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889576&type=01&language=es_ES',
 'https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=889577&type=01&

# DOUE

In [189]:
# --------------------------------------------------------------------------------------
# DOUE
# --------------------------------------------------------------------------------------

URL_HTML_resumen =  "https://eur-lex.europa.eu/oj/direct-access.html?locale=es"

## carga página HTML y genera árbol

response = requests.get(URL_HTML_resumen)
tree = html.fromstring(response.text)

## Recoge Nombre Secciones Sumario

secciones = tree.xpath('//*[@id="MainContent"]/div[2]/div[2]/div/table/tbody/tr[1]')
#print(secciones)



In [190]:
## Recoge Valores para formar URLs Secciones Sumario

df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion_name = seccion.xpath('./td[2]/a/text()')
    seccion_URL = seccion.xpath('./td[2]/a/@href')
    #seccion = re.sub('(\\r|\\n|\\t)+', '', seccion)
    #print(seccion_name)
    #print(seccion_URL)

for row in range(len(seccion_name)):
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion_name[row],
                                            'Seccion_link' : 'https://eur-lex.europa.eu' + seccion_URL[row][4:]},
                                            ignore_index=True)

In [192]:
df_secciones_sumarios

,Seccion,Seccion_link
0,L436,https://eur-lex.europa.eu/legal-content/ES/TXT...
1,L437,https://eur-lex.europa.eu/legal-content/ES/TXT...
2,L438,https://eur-lex.europa.eu/legal-content/ES/TXT...


In [191]:
[ print (row) for row in df_secciones_sumarios['Seccion_link']]

https://eur-lex.europa.eu/legal-content/ES/TXT/?uri=OJ:L:2020:436:TOC
https://eur-lex.europa.eu/legal-content/ES/TXT/?uri=OJ:L:2020:437:TOC
https://eur-lex.europa.eu/legal-content/ES/TXT/?uri=OJ:L:2020:438:TOC


[None, None, None]

In [193]:
df_sumarios = pd.DataFrame()

for URL in df_secciones_sumarios['Seccion_link']:
    
    ### Recoge Items en Seccion Legislación

    #print(URL)
    response = requests.get(URL)
    sumario_HTML = html.fromstring(response.text)

    documentos = sumario_HTML.xpath('//*[@class="oj-ti-doc-dur"]/a[1]/text()[1]')
    documentos_URL = sumario_HTML.xpath('//*[@class="oj-ti-doc-dur"]/a[1]/@href')

    for row in range(len(documentos)):
        df_sumarios = df_sumarios.append({'Seccion': 'L' + documentos_URL[row][-7:-4],
                                        'Item_Name': documentos[row],
                                        'Item_Link': 'https://eur-lex.europa.eu' + documentos_URL[row][10:],
                                        'PDF_Link': ''},
                                        ignore_index=True)


In [196]:
df_sumarios

,Seccion,Item_Name,Item_Link,PDF_Link
0,L436,Reglamento Delegado (UE) 2020/2237 de la Comis...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
1,L436,Decisión de Ejecución (UE) 2020/2238 de la Com...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
2,L436,Decisión de Ejecución (UE) 2020/2239 de la Com...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
3,L436,Decisión de Ejecución (UE) 2020/2240 de la Com...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
4,L436,Decisión de Ejecución (UE) 2020/2241 de la Com...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
5,L436,Decisión de Ejecución (UE) 2020/2242 de la Com...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
6,L436,Recomendación (UE) 2020/2243 de la Comisión de...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
7,L437,Reglamento (UE) 2020/2220 del Parlamento Europ...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
8,L437,Reglamento (UE) 2020/2221 del Parlamento Europ...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
9,L437,Reglamento (UE) 2020/2222 del Parlamento Europ...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...


In [194]:
row = 0
for link in df_sumarios['Item_Link']:

    ### Recoge PDF link en Items Legislación

    #print(URL)
    response = requests.get(link)
    sumario_HTML = html.fromstring(response.text)

    PDF_Link = sumario_HTML.xpath('//*[@id="format_language_table_PDF_ES"]/@href')
    df_sumarios['PDF_Link'].iloc[row] = 'https://eur-lex.europa.eu' + PDF_Link[0][10:]

    row += 1
    print(PDF_link)

['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-content/ES/TXT/PDF/?uri=CELEX:32020D2189&from=ES']
['./../../../legal-c

In [195]:
df_sumarios = df_sumarios[['Seccion', 'Item_Name', 'Item_Link', 'PDF_Link']]
df_sumarios

,Seccion,Item_Name,Item_Link,PDF_Link
0,L436,Reglamento Delegado (UE) 2020/2237 de la Comis...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
1,L436,Decisión de Ejecución (UE) 2020/2238 de la Com...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
2,L436,Decisión de Ejecución (UE) 2020/2239 de la Com...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
3,L436,Decisión de Ejecución (UE) 2020/2240 de la Com...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
4,L436,Decisión de Ejecución (UE) 2020/2241 de la Com...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
5,L436,Decisión de Ejecución (UE) 2020/2242 de la Com...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
6,L436,Recomendación (UE) 2020/2243 de la Comisión de...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
7,L437,Reglamento (UE) 2020/2220 del Parlamento Europ...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
8,L437,Reglamento (UE) 2020/2221 del Parlamento Europ...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...
9,L437,Reglamento (UE) 2020/2222 del Parlamento Europ...,https://eur-lex.europa.eu/legal-content/ES/AUT...,https://eur-lex.europa.eu/legal-content/ES/TXT...




## Recoge Valores para formar URLs Secciones Sumario

URL_base_sumario = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

df_URL_sumarios = pd.DataFrame()
for seccion in secciones:    
    cadena = ''
    for input in seccion.xpath('./form'):
        argumentos = input.xpath('./input/@name')
        valores = input.xpath('./input/@value')
        #print(argumentos, valores)

    for indice in range(len(argumentos)):
        cadena += argumentos[indice] + '=' + valores[indice] + '&'

    URL_sumario = 'https://dogc.gencat.cat' + str(URL_base_sumario[0]) + '?' + str(cadena[:-1])
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

## Concatena en df columnas Sccion y URL de sumarios 

df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

### Recoge Items en Seccion Disposiciones

response = requests.get(df_sumarios['URL_Seccion'][0])
sumario_HTML = html.fromstring(response.text)

#seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div[1]/text()')
seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div/text()')
bloques = sumario_HTML.xpath('//*[@id="disposicions"]')
df_disposiciones = pd.DataFrame(columns=['Seccion','Item_Name','PDF_link'])
for bloque in bloques: 
    item_name = bloque.xpath('./div/p/text()')
    #item_name = re.sub('(\\r|\\n|\\t)+', '', item_name[0])
    pdf_link = bloque.xpath('./div/div/a[4]/@href')

for row in range(len(pdf_link)):
    df_disposiciones = df_disposiciones.append({'Item_Name': re.sub('(\\r|\\n|\\t)+', '', item_name[row]),
                                            'PDF_link' : pdf_link[row],
                                            'Seccion' : seccion[0][:-1]},
                                            ignore_index=True)

#df_disposiciones

def modifica_tabla_resultados_DOGC(tabla):
    tabla = tabla.reset_index(drop=True)
    tabla['Item_Name'] = '<a href=' + tabla['PDF_link'] + ' ' + 'target="_blank"' + '><div>' + tabla['Item_Name'] + '</div></a>' # Añade url link  pdf a Item_id
    tabla.drop(['PDF_link'], axis='columns', inplace=True)
    df_html = tabla.reset_index(drop=True).to_html(index='True', classes="table-hover", escape=False) # Utiliza Clase table-hover de Bootstrap
    df_html = df_html.replace("dataframe", "")  # Elimina clase por defecto dataframe
    df_html = df_html.replace('border="1"', 'border="2"')  # Incrementa tamaño línea borde tabla
    df_html = df_html.replace("<table", '<table style="font-size:12px; text-align: center; width: 100%" ') # Cambia tamaño fuente a 15px
    df_html = df_html.replace("<th>"+ tabla.columns[0], '<th style="text-align: center">'+ tabla.columns[0]) # Cambia alineación a header Columna 1
    df_html = df_html.replace("<th>"+ tabla.columns[1], '<th style="text-align: center">'+ tabla.columns[1]) # Cambia alineación a header Columna 2
    
    return df_html

st.header('')
st.header('DOGC disposiciones')


tabla_resultados_presentacion_DOGC = modifica_tabla_resultados_DOGC(df_disposiciones)

st.write(tabla_resultados_presentacion_DOGC, unsafe_allow_html=True)